In [ ]:
# check gpu connection

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# connect to drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pip install pdfplumber

In [ ]:
import os
import pdfplumber
import pandas as pd
import time
import numpy as np
import re

In [ ]:
# This block converts pdf's into pandas dataframe as a string
start = time.time()

def get_filepaths(directory):
  
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.


In [ ]:
# Find case_id
def find_case_id(file_array,file_index):
  with tf.device('/device:GPU:0'):  
    name = (file_array[file_index].split('/')[6])
    case_id_temp = name.split('-')
    case_id = case_id_temp[2] + "-" + case_id_temp[3]
    case_problem_identifier = case_id_temp[4][0]

    return case_id,case_problem_identifier

# Loop through all pages in pdf file
# **ADDING Extracting the judges name from the first page
def loop(file):
  with tf.device('/device:GPU:0'):  
    text = ""
    for page in range(len(file.pages)):
            text += file.pages[page].extract_text()

    return text 

In [ ]:

# run with gpu
with tf.device('/device:GPU:0'):  

  # court that we want to convert to csv
  court_nums = [1,2,3,4,5,6,7,8,9,10,11,12]

  for court in court_nums:


    files = get_filepaths("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(court))


    # randomly select a subset of 1000 files
    files = np.random.choice(files, 1000, replace= False)

    data = pd.DataFrame(columns=['Court_id','Case_id',"Opinion"])   #Added judges here - took out

    count = 0
    completed_count = 0

    # loop through all files in directory
    for file in files:
      if completed_count == 500:
          break;
      with tf.device('/device:GPU:0'):  
        with pdfplumber.open(file) as f:
            case_id, case_problem_id = find_case_id(count)
            if len(f.pages) < 2 or case_problem_id == 1:
              count+=1
              print(count)
              continue
            all_text = loop(f)
            data.loc[len(data.index)] = [court, case_id ,all_text]
        completed_count +=1
        count+=1
        # keep track of where we are
        print(count)
        # In case process crashes we have some data that is cleaned, so we don't need to completely restart
        data.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(court)+".csv")
    
    # Convert data to csv
    data.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(court)+".csv")


    end = time.time()
    print(end-start)